In [7]:
import joblib
import pandas as pd
from datetime import datetime, timedelta
import requests

# Tải mô hình và các đối tượng
def load_model_and_objects():
    model = joblib.load("../models/weather_rf_model.joblib")
    scaler = joblib.load("../models/weather_scaler.joblib")
    label_encoder = joblib.load("../models/weather_label_encoder.joblib")
    with open("../models/features.txt", "r") as f:
        features = f.read().splitlines()
    return model, scaler, label_encoder, features

# Lấy dữ liệu thời tiết hiện tại từ API
def get_current_weather(lat, lon, api_key):
    url = "http://api.weatherapi.com/v1/current.json"
    params = {"key": api_key, "q": f"{lat},{lon}"}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return {
            "timestamp": data["location"]["localtime"],
            "temperature": data["current"]["temp_c"],
            "humidity": data["current"]["humidity"],
            "wind_speed": data["current"]["wind_kph"],
            "pressure": data["current"]["pressure_mb"],
            "cloud": data["current"]["cloud"],
            "condition": data["current"]["condition"]["text"],
        }
    else:
        print("Error fetching weather data:", response.status_code)
        return None

# Dự đoán thời tiết sau 3h, 6h, 12h
def predict_weather(model, scaler, features, current_data):
    predictions = []
    base_time = datetime.strptime(current_data["timestamp"], "%Y-%m-%d %H:%M")
    
    # Add missing features with default values
    for feature in features:
        if feature not in current_data:
            current_data[feature] = 0  # Default value, you can adjust as needed
    
    input_data = pd.DataFrame([current_data])[features]
    input_data_scaled = scaler.transform(input_data)

    for hours in [3, 6, 12]:
        prediction = model.predict(input_data_scaled)[0]
        prediction_time = base_time + timedelta(hours=hours)
        predictions.append({"time": prediction_time.strftime("%Y-%m-%d %H:%M"), "prediction": prediction})
    
    return predictions

# Chạy chương trình
def main():
    model, scaler, label_encoder, features = load_model_and_objects()
    api_key = "a5b32b6e884e4b5aa5b95910241712"
    lat, lon = 21.0285, 105.8542  # Hà Nội
    
    current_weather = get_current_weather(lat, lon, api_key)
    if not current_weather:
        print("Failed to fetch current weather data!")
        return
    
    predictions = predict_weather(model, scaler, features, current_weather)
    print("Current Weather:", current_weather)
    print("Predicted Weather:")
    for pred in predictions:
        print(f"Time: {pred['time']}, Prediction: {pred['prediction']}")

if __name__ == "__main__":
    main()


Current Weather: {'timestamp': '2025-01-01 16:47', 'temperature': 23.2, 'humidity': 47, 'wind_speed': 5.0, 'pressure': 1014.0, 'cloud': 0, 'condition': 'Sunny', 'hour': 0, 'day': 0, 'month': 0, 'day_of_week': 0, 'airport_code_encoded': 0, 'latitude': 0, 'longitude': 0, 'temp_lag_1': 0, 'temp_lag_3': 0, 'humidity_lag_1': 0, 'temp_rolling_mean_6h': 0, 'humidity_rolling_mean_6h': 0, 'wind_direction_encoded': 0, 'visibility': 0, 'precipitation': 0, 'condition_encoded': 0, 'wind_speed_lag_1': 0, 'wind_speed_lag_3': 0, 'pressure_lag_1': 0, 'pressure_lag_3': 0, 'cloud_lag_1': 0, 'cloud_lag_3': 0, 'visibility_lag_1': 0, 'visibility_lag_3': 0, 'precipitation_lag_1': 0, 'precipitation_lag_3': 0}
Predicted Weather:
Time: 2025-01-01 19:47, Prediction: [   9.78897843   47.            5.            0.         1014.
    0.            0.            0.            0.            0.
   -2.80510664    1.84932857    0.            5.68067381]
Time: 2025-01-01 22:47, Prediction: [   9.78897843   47.          